# An Example 2D optimisation
## Inspired by the following notebook:
[this blog post](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/) referring to 
[this notebook](https://github.com/thuijskens/bayesian-optimization/blob/master/ipython-notebooks/svm-optimization.ipynb)

target function and inspiration for plotting from [here](https://github.com/fmfn/BayesianOptimization/issues/18)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import sklearn.gaussian_process as gp
import matplotlib.pyplot as plt
import seaborn as sns # prettify matplotlib

In [ ]:
# local modules
import sys; sys.path.append('../src/')
import plot3D
import optimisation as op
import optimisation_gui as op_gui
import synthetic_data

In [ ]:
# Make deterministic
np.random.seed(42)

# Target Function

In [ ]:
x = y = np.linspace(0, 6, 300)
X, Y = np.meshgrid(x, y)
noise = synthetic_data.Noise2D(x, y, 0.1)

# vectorize needed for accessing noise
#@np.vectorize
def target(x, y):
    ''' from https://github.com/fmfn/BayesianOptimization/issues/18 '''
    a = np.exp(-( (x - 2)**2/0.7 + (y - 4)**2/1.2) + (x - 2)*(y - 4)/1.6 )
    b = np.exp(-( (x - 4)**2/3 + (y - 2)**2/2.) )
    c = np.exp(-( (x - 4)**2/0.5 + (y - 4)**2/0.5) + (x - 4)*(y - 4)/0.5 )
    d = np.sin(3.1415 * x)
    e = np.exp(-( (x - 5.5)**2/0.5 + (y - 5.5)**2/.5) )
    return 2*a + b - c + 0.17 * d + 2*e + noise.get(x, y)

Z = target(X, Y)
best_z = np.max(Z) # ~2.06

class TestEvaluator(op.Evaluator):
    def test_config(self, config):
        return target(config.x, config.y)
evaluator = TestEvaluator()

plot3D.surface3D(X, Y, Z)

# Helper Functions

In [ ]:
gp_params = {
    'alpha': 1e-10, # default noise level, deal with noise with the white kernel
    'kernel': 1.0 * gp.kernels.Matern(nu=1.5) + gp.kernels.WhiteKernel(),
    'n_restarts_optimizer': 4
}
ac_params = {
    'kappa' : 5
}
ranges = {'x' : x, 'y' : y}

bo = op.BayesianOptimisationOptimiser(
    ranges, maximise_cost=True,
    acquisition_function='UCB', acquisition_function_params=ac_params,
    gp_params=gp_params, pre_samples=4, ac_max_params=None, close_tolerance=1e-4)

In [ ]:
bo.run_sequential(evaluator, max_jobs=60)

# or run interactively to see the log as it runs
#task = lambda: bo.run_sequential(evaluator, max_jobs=60)
#op_gui.interactive(bo, task)

In [ ]:
bo.plot_cost_over_time(true_best=best_z);

In [ ]:
def plot(n, step):
    bo.plot_step_slice('x', n, gp_through_all=True)
op_gui.step_log_slider(bo, plot);

In [ ]:
def plot2(n, step):
    bo.plot_step_slice('y', n, gp_through_all=True)
op_gui.step_log_slider(bo, plot2);

In [ ]:
num_random = bo.num_randomly_chosen()
num_bayes = len(bo.samples) - num_random
print('number of random samples: {}'.format(num_random))
print('number of bayes samples: {}'.format(num_bayes))

In [ ]:
def plot2D(n, step):
    bo.plot_step_2D('x', 'y', n, true_cost=Z)
op_gui.step_log_slider(bo, plot2D, pre_compute=False);

In [ ]:
bo.scatter_plot('x', 'y')

# Try optimising the same function with random search

In [ ]:
ra = op.RandomSearchOptimiser(ranges, maximise_cost=True)
ra.run_sequential(evaluator, max_jobs=1000)

In [ ]:
ra.plot_cost_over_time(true_best=best_z);

In [ ]:
ra.scatter_plot('x', 'y')